1. Flatten the primary annotations and their closures in to a single list for each disease
2. Do termset comparison of the query disease to each of the other diseases and calculates simple jaccard index.
3. Sort the results by jaccard index and return the top 10 results.

In [208]:
import pandas as pd
import time
import math
from oaklib.datamodels.vocabulary import OWL_THING
from oaklib import get_adapter
from oaklib.types import CURIE, PRED_CURIE
from oaklib.interfaces.obograph_interface import OboGraphInterface
from oaklib.interfaces.basic_ontology_interface import BasicOntologyInterface
from oaklib.interfaces.semsim_interface import SemanticSimilarityInterface
from typing import Iterable, Iterator, List, Optional, Tuple

from oaklib.implementations.ubergraph.ubergraph_implementation import UbergraphImplementation
from oaklib.resource import OntologyResource

## Get D2P Associations from Monarch Edges

In [210]:
edges = pd.read_csv('monarch-kg/monarch-kg_edges.tsv', sep='\t', dtype=str)

In [211]:
disease_phenotypes = edges[(edges['category'] == 'biolink:DiseaseToPhenotypicFeatureAssociation') & (edges['subject'].str.contains("MONDO"))]

In [212]:
d2p_annotations = disease_phenotypes[['subject', 'object']]
d2p_annotations.head()

,subject,object
2869978,MONDO:0023659,HP:0011097
2869979,MONDO:0023659,HP:0002187
2869980,MONDO:0023659,HP:0001518
2869981,MONDO:0023659,HP:0032792
2869982,MONDO:0023659,HP:0011451


## Get Inferred subClassOf relations from Phenio for all HP terms

In [213]:
phen_rel = pd.read_csv('phenio-relation-graph.tsv', sep='\t', dtype=str)

In [214]:
phen_rel.columns = ['subject', 'predicate', 'object']

In [215]:
pr_isa = phen_rel[(phen_rel['predicate'] == 'rdfs:subClassOf') & (phen_rel['subject'].str.startswith('HP:')) & (phen_rel['object'].str.startswith('HP:'))]

## Join the D2P annotations to the inferred subClassOf relations

In [216]:
merged_d2p = pd.merge(left=d2p_annotations, right=pr_isa, how='left', left_on='object', right_on='subject')
merged_d2p.head()

,subject_x,object_x,subject_y,predicate,object_y
0,MONDO:0023659,HP:0011097,HP:0011097,rdfs:subClassOf,HP:0000001
1,MONDO:0023659,HP:0011097,HP:0011097,rdfs:subClassOf,HP:0011097
2,MONDO:0023659,HP:0011097,HP:0011097,rdfs:subClassOf,HP:0000118
3,MONDO:0023659,HP:0011097,HP:0011097,rdfs:subClassOf,HP:0020219
4,MONDO:0023659,HP:0011097,HP:0011097,rdfs:subClassOf,HP:0000707


## Flatten the annotations and their closures in to a single list for each disease

In [217]:
expanded_diseases = merged_d2p.groupby('subject_x')['object_y'].apply(set).reset_index()

## Add labels to the diseases

In [218]:
handle = get_adapter("sqlite:obo:mondo")
expanded_diseases['subject_label'] = expanded_diseases['subject_x'].apply(lambda x: handle.label(x))

## Jaccard search

In [219]:
def jaccard_index(row, query_set):
    set1 = row['object_y']
    set2 = query_set
    intersection = len(set(set1).intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

def get_label(id):

    return adapter.get_label(id)

## Create a random disease profile

In [227]:
rand_disease = expanded_diseases.sample(1)
rand_disease_profile = rand_disease['object_y'].iloc[0]
rand_disease_profile
# example disease MONDO:0030017
mondo_0030017 = {'HP:0000001',
 'HP:0000118',
 'HP:0000707',
 'HP:0000924',
 'HP:0001252',
 'HP:0001319',
 'HP:0001367',
 'HP:0001384',
 'HP:0001507',
 'HP:0001510',
 'HP:0001511',
 'HP:0001518',
 'HP:0001871',
 'HP:0001939',
 'HP:0001941',
 'HP:0002011',
 'HP:0002013',
 'HP:0002017',
 'HP:0002020',
 'HP:0002086',
 'HP:0002151',
 'HP:0002188',
 'HP:0002587',
 'HP:0002644',
 'HP:0002795',
 'HP:0002813',
 'HP:0002814',
 'HP:0002867',
 'HP:0003011',
 'HP:0003170',
 'HP:0003236',
 'HP:0003272',
 'HP:0003287',
 'HP:0003808',
 'HP:0004323',
 'HP:0004325',
 'HP:0004360',
 'HP:0004364',
 'HP:0005262',
 'HP:0008347',
 'HP:0008807',
 'HP:0008872',
 'HP:0008972',
 'HP:0010876',
 'HP:0011017',
 'HP:0011021',
 'HP:0011024',
 'HP:0011400',
 'HP:0011458',
 'HP:0011804',
 'HP:0011842',
 'HP:0011844',
 'HP:0011922',
 'HP:0011923',
 'HP:0011924',
 'HP:0011968',
 'HP:0012103',
 'HP:0012337',
 'HP:0012415',
 'HP:0012447',
 'HP:0012448',
 'HP:0012639',
 'HP:0012719',
 'HP:0025031',
 'HP:0025032',
 'HP:0025270',
 'HP:0025354',
 'HP:0032180',
 'HP:0033127',
 'HP:0040064',
 'HP:0040068',
 'HP:0040069',
 'HP:0040081',
 'HP:0100491',
 'HP:0500165'}

In [221]:
rand_disease[['subject_label', 'subject_x']]

,subject_label,subject_x
8613,combined oxidative phosphorylation deficiency 43,MONDO:0030017


## Search for similar diseases using pandas apply

In [228]:
start = time.time()
expanded_diseases['jaccard'] = expanded_diseases.apply(jaccard_index, query_set=mondo_0030017, axis=1)
end = time.time()
print(end-start)
expanded_diseases.sort_values('jaccard', ascending=False).head(10)[['subject_label', 'subject_x', 'jaccard']]

0.6687829494476318


,subject_label,subject_x,jaccard
8613,combined oxidative phosphorylation deficiency 43,MONDO:0030017,1.000000
2379,myopathy with abnormal lipid metabolism,MONDO:0009703,0.339450
9083,"mitochondrial complex 1 deficiency, nuclear ty...",MONDO:0032635,0.330357
4515,combined oxidative phosphorylation defect type 2,MONDO:0012510,0.323741
9076,"mitochondrial complex 1 deficiency, nuclear ty...",MONDO:0032628,0.320000
9077,"mitochondrial complex 1 deficiency, nuclear ty...",MONDO:0032629,0.318681
8361,combined oxidative phosphorylation deficiency 22,MONDO:0020727,0.316239
9075,"mitochondrial complex 1 deficiency, nuclear ty...",MONDO:0032627,0.303922
9395,multiple mitochondrial dysfunctions syndrome 5,MONDO:0033282,0.303279
6164,mitochondrial complex III deficiency nuclear t...,MONDO:0014496,0.300885


## Search for similar diseases using a for loop

In [223]:
start = time.time()
scores = []
for index, row in expanded_diseases.iterrows():
    scores.append(jaccard_index(row, query_set=rand_disease_profile))
end = time.time()
print(end-start)

0.7271339893341064


In [226]:
len(rand_disease_profile)

75

In [224]:
oi = UbergraphImplementation()

In [225]:
start = time.time()
for ph in rand_disease_profile:
    ic = oi.get_information_content(ph)
    print(ic)
end = time.time()
print(end-start)

3.04059996e-37
3.15162319e-26
-1.98508264e-23
1.63806185e-40
100.0
1.89813657e-36
3.41089861e-35
4.53060232e-35
0.0
-1.55273663e-36
-1.04565136e-31
0.0
0.0
100.0
0.0
0.0
-6.61774151e-23
2.18062173e-33
-2.61261948e-06
5.12845678e-36
100.0
2.06913564e-14
39934958200000.0
1.89813657e-36
6244596.5
-6.9949896e-29
1.89813657e-36
100.0
0.0
-64587319700000.0
-194913.062
0.0
4.11030533e-06
0.249956116
-1.36959142e-11
-1.35482623e-14
0.0
1.31732527e-31
-6.14219105e-39
100.0
-5.72592884e-35
100.0
-12876552200.0
1.39171549e-37
294265289000000.0
1.89813657e-36
-199707658000000.0
0.0
-1.27000214e-29
0.0
-1.55273663e-36
1.35278273
-1.35482623e-14
0.0
3.5024704e-38
100.0
3.41089861e-35
-21.8520203
0.0
0.00228426699
1.87962189e-20
-3012681.5
100.0
-1.30881371e-15
7.52309357e-22
0.0
0.0
0.0
2.06913564e-14
1.92692552e-41
1.62530017e-19
0.0
1.13568092e-07
2.61482293e-42
-1.30881371e-15
31.674468278884888
